In [1]:
#----------------------------------------------------------------------
# (1) Clean environment and install packages
#----------------------------------------------------------------------
rm(list = ls())                         #Clean Memory
# install.packages("<package name>")    # to install missing packages

In [ ]:
#----------------------------------------------------------------------
# (1) Load Packgages
#----------------------------------------------------------------------
require(tidyverse)
require(broom)
require(xts)
require(lmtest)
require(sandwich)
source("Helper_Functions_.R")

# overide tidyverse loading of select function
 select = dplyr::select

In [ ]:
#----------------------------------------------------------------------
# (2) Load Datasets
#----------------------------------------------------------------------
load("./Datasets/dataset.RData")

### Asset class levels
Asset_Class_L = c("Eq_Ex_Fin", "Eq_Ind_Adj", "Industries", "Commodities", "Currencies", "Bonds",  "Equity_Indexes")
 
# Risk-Free
RF <- read_csv("./Datasets/FF_RF.csv") %>% 
  dplyr::select(Date, RF) %>% 
  as.matrix %>%
  xts(x = .[ , 2], order.by = as.yearmon(as.character(.[ , 1]), format = "%Y%m")) %>% 
  myrename(., "RF") %>% .[, 1]/100


## Subset datasets  
datasets %<>% mutate(Asset_Class = factor(Asset_Class, levels = Asset_Class_L),
                     Wghting = factor(Wghting, levels = c("Rank", "Sort"))) %>%
  mutate(RF = RF %>% list)  %>% 
  mutate(data = map(data, ~.x %>% filter(Date >= "Jan 1972" & Date <= "Dec 2017")))

horizons = c(1, 12, 24)

### (3) Table 2

In [4]:
#----------------------------------------------------------------------
# (3) Panel A: Summary Statistics
#----------------------------------------------------------------------

### Summary statistics function
SummaryPort_fn = function(x){
  mytbl <- x %>% 
    as.data.frame() %>% 
    summarize(Mean = mean(x), Std = sd(x), t.stat = t.test(x)$statistic,
              Sharpe = mean(x)/sd(x)) %>% 
    mutate(Mean = formatN2(Mean * 100, 2),
           t.stat = formatN2(t.stat, 2),
           Sharpe = formatN2(Sharpe, 2))
  return(mytbl)
}


unq_reg = datasets %>% 
  group_by(Asset_Class, Wghting) %>%
  summarise(results   = map(data, ~.x %>%
                                      mutate(Std      = (ret_High - ret_Low) %>% sd,
                                             StdRet01 = ret_High/Std*0.15/sqrt(12),
                                             StdRet02 = ret_Low/Std*0.15/sqrt(12),
                                             ydk      = StdRet01 - StdRet02) %>%
                                     pull(ydk) %>%
                                     SummaryPort_fn(.))) %>%
  unnest(results) %>%  
  group_by(Asset_Class, Wghting) %>%
  select("Wghting", "Asset_Class", "Mean",  "t.stat", "Sharpe") %>%
  arrange(Wghting, Asset_Class) %>% 
  ungroup

unq_reg = bind_cols(filter(unq_reg, Wghting == "Sort") %>% 
                      select(Asset_Class, Mean, t.stat, Sharpe),
                    filter(unq_reg, Wghting == "Rank")%>% 
                      select(Asset_Class, Mean, t.stat, Sharpe))

print("Panel A")
unq_reg

[1] "Panel A"


Asset_Class,Mean,t.stat,Sharpe,Asset_Class1,Mean1,t.stat1,Sharpe1
<fct>,<chr>,<chr>,<chr>,<fct>,<chr>,<chr>,<chr>
Eq_Ex_Fin,0.15,0.83,0.04,Eq_Ex_Fin,0.21,1.11,0.05
Eq_Ind_Adj,0.32,1.73,0.07,Eq_Ind_Adj,0.49,2.65,0.11
Industries,0.04,0.19,0.01,Industries,0.05,0.25,0.01
Commodities,0.30,1.60,0.07,Commodities,0.27,1.48,0.06
Currencies,0.41,2.13,0.09,Currencies,0.46,2.38,0.11
Bonds,-0.04,-0.16,-0.01,Bonds,-0.05,-0.20,-0.01
Equity_Indexes,0.22,0.87,0.05,Equity_Indexes,0.32,1.27,0.08


In [ ]:
#----------------------------------------------------------------------
# (4) Panel B: Time Series Regressions
#----------------------------------------------------------------------

### Time Series regression functions
# Newey weest
reg_fn_nw = function(x, y, z, RF){
  x   <- ComRet(R = xts(x[, "High"], order.by = x[, "Date"]), RF, H = z) - 
         ComRet(R = xts(x[, "Low"],  order.by = x[, "Date"]), RF, H = z)
  x <- x[!is.na(x)]
  y <-  scale(y[1:length(x)])
  reg <-  lm(x ~ y)
  reg.summ <- summary(reg)
  reg.summ$coefficients <- unclass(coeftest(reg, vcov. = olsNW02(reg, lag = z)))
  
  res <-  c(formatN2(reg.summ$coefficients[1,1] * 100, 2), formatN2(reg.summ$coefficients[2,1] * 100, 2),
            formatN2(reg.summ$coefficients[1,3], 2), formatN2(reg.summ$coefficients[2,3], 2),
            formatN2(I(reg.summ$adj.r.squared*100), 2))
  names(res) <- c("a", "b", "t.a", "t.b", "R")
  return(as.data.frame(t(res)))
}

reg_fn_hd = function(x, y, z, RF){
  df01 = x[ ,"High"] - x[ ,"Low"]
  VCV = hodrickse(lm(df01[!is.na(df01)] ~ scale(y[1:length(df01[!is.na(df01)])])), lags = z)
  x   <- ComRet(R = xts(x[, "High"], order.by = x[, "Date"]), RF, H = z) - 
    ComRet(R = xts(x[, "Low"], order.by = x[, "Date"]), RF, H = z)
  x <- x[!is.na(x)]
  y <-  scale(y[1:length(x)])
  reg <-  lm(x ~ y)
  reg.summ <- summary(reg)
  reg.summ$coefficients <- unclass(coeftest(reg, vcov. = VCV))
  
  res <-  c(formatN2(reg.summ$coefficients[1,1] * 100, 2), formatN2(reg.summ$coefficients[2,1] * 100, 2),
            formatN2(reg.summ$coefficients[1,3], 2), formatN2(reg.summ$coefficients[2,3], 2),
            formatN2(I(reg.summ$adj.r.squared*100), 2))
  names(res) <- c("a", "b", "t.a", "t.b", "R")
  return(as.data.frame(t(res)))
}

# Hodrick

reg_fn_02 = function(x, y, RF){
  tibble(results = map(horizons, ~reg_fn_nw(x = x, y = y, z = .x, RF = RF)),
         H = as.character(horizons)) %>% unnest
}

reg_fn_03 = function(x, y, RF){
  tibble(results = map(horizons, ~reg_fn_hd(x = x, y = y, z = .x, RF = RF)),
         H = as.character(horizons)) %>% unnest
}

 
data = datasets %>% 
  ungroup %>%
  mutate_at(vars(Asset_Class, Wghting), ~as.character(.)) %>%
  group_by(Asset_Class, Wghting) %>%
  mutate(data = map(data, ~.x %>% mutate(Std      = (ret_High - ret_Low) %>% sd,
                                         StdRet01 = ret_High/Std*0.15/sqrt(12),
                                         StdRet02 = ret_Low/Std*0.15/sqrt(12)))) %>%
  transmute(Reg_nw = map2(data, RF, ~reg_fn_02(x = data.frame(High = .x$StdRet01, Low = .x$StdRet02, Date = .x$Date), 
                                               y = .x$Cha_Spread, RF = .y)),
            Reg_hd = map2(data, RF, ~reg_fn_03(x = data.frame(High = .x$StdRet01, Low = .x$StdRet02, Date = .x$Date), 
                                               y = .x$Cha_Spread, RF = .y))) %>%
  gather(Series, Values, -Asset_Class, -Wghting) %>%
  unnest %>% 
  group_by(Series) %>% 
  mutate(StdErr = str_split(Series, "_")[[1]][2]);

In [6]:
unq_reg = data %>% 
  ungroup

unq_reg = bind_cols(filter(unq_reg, Wghting == "Sort" & StdErr == "nw") %>% select(Asset_Class, H, a, b, t.a, t.b),
                    filter(unq_reg, Wghting == "Sort" & StdErr == "hd") %>% select(t.a, t.b, R), 
                    filter(unq_reg, Wghting == "Rank" & StdErr == "nw") %>% select(a, b, t.a, t.b),
                    filter(unq_reg, Wghting == "Rank" & StdErr == "hd") %>% select(t.a, t.b))

print("Panel B")
unq_reg

[1] "Panel B"


Asset_Class,H,a,b,t.a,t.b,t.a1,t.b1,R,a1,b1,t.a2,t.b2,t.a3,t.b3
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Eq_Ex_Fin,1,0.15,0.48,0.80,2.35,0.84,2.20,1.03,0.21,0.35,1.06,1.53,1.11,1.52
Eq_Ex_Fin,12,2.01,6.97,0.96,3.89,0.92,3.24,13.94,3.01,5.54,1.30,2.30,1.38,2.45
Eq_Ex_Fin,24,4.49,16.96,1.09,4.88,1.04,4.24,30.86,7.19,15.62,1.53,2.99,1.65,3.66
Eq_Ind_Adj,1,0.32,0.65,1.65,2.76,1.73,2.75,2.09,0.49,0.54,2.52,2.02,2.65,2.04
Eq_Ind_Adj,12,4.59,9.72,2.22,4.62,2.10,4.17,26.24,7.28,8.79,2.87,2.91,3.33,3.59
Eq_Ind_Adj,24,9.98,22.46,2.24,4.21,2.31,5.31,39.95,16.94,22.35,3.22,3.36,3.89,4.93
Industries,1,0.04,0.20,0.19,1.07,0.19,1.00,0.03,0.05,0.18,0.24,0.87,0.25,0.83
Industries,12,0.29,4.34,0.15,2.58,0.13,2.09,6.44,0.41,3.67,0.18,1.78,0.19,1.65
Industries,24,-0.10,13.17,-0.02,3.08,-0.02,3.39,16.86,-0.02,11.73,-0.00,2.06,-0.00,2.86
